In [1]:
%pip install tensorflow scikit-learn

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   -----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ell-ai 0.0.12 requires numpy<3.0.0,>=2.0.1, but you have numpy 1.26.4 which is incompatible.
onnx 1.15.0 requires protobuf>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.5 which is incompatible.
streamlit 1.39.0 requires protobuf<6,>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorboardx 2.6.2.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2

In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set paths
data_dir = 'dataset'
batch_size = 32
img_height = 224
img_width = 224

# ImageDataGenerator for data loading and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # 80% training, 20% validation split
)

# Load train and validation data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

# Define the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# After training, unfreeze some layers in the base model for fine-tuning
for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Test set loading
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_data = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the model in HDF5 format
model.save('pest_identification_model.h5')

# Save the model in TensorFlow SavedModel format
model.save('pest_identification_model')


Found 2175 images belonging to 12 classes.
Found 542 images belonging to 12 classes.
Epoch 1/20
68/68 [==============================] - 336s 5s/step - loss: 2.5743 - accuracy: 0.1214 - val_loss: 2.5099 - val_accuracy: 0.1310
Epoch 2/20
68/68 [==============================] - 318s 5s/step - loss: 2.4849 - accuracy: 0.1343 - val_loss: 2.5065 - val_accuracy: 0.1199
Epoch 3/20
68/68 [==============================] - 319s 5s/step - loss: 2.4194 - accuracy: 0.1513 - val_loss: 2.4955 - val_accuracy: 0.1531
Epoch 4/20
68/68 [==============================] - 319s 5s/step - loss: 2.3985 - accuracy: 0.1554 - val_loss: 2.4237 - val_accuracy: 0.1476
Epoch 5/20
68/68 [==============================] - 316s 5s/step - loss: 2.3562 - accuracy: 0.1821 - val_loss: 2.4452 - val_accuracy: 0.1255
Epoch 6/20
68/68 [==============================] - 284s 4s/step - loss: 2.3363 - accuracy: 0.1963 - val_loss: 2.4205 - val_accuracy: 0.1476
Epoch 7/20
68/68 [==============================] - 306s 5s/step - lo

ValueError: Asked to retrieve element 0, but the Sequence has length 0